# Particle Surface Tutorial

Understanding how particle surfaces are represented is crucial in the study of condensation processes involving water and organic molecules. This is primarily influenced by the Kelvin curvature effect, which describes how the saturation vapor pressure of a liquid droplet varies with its size. This tutorial will introduce the fundamental approaches to modeling the particle surface in aerosol particle simulations.

[Kelvin Curvature Effect](https://en.wikipedia.org/wiki/Kelvin_equation)

## Strategies for Surface Representation

To accurately simulate particle surfaces, one must adhere to the `SurfaceStrategy` abstract base class. This class outlines common methods required for all surface representation strategies:

- `kelvin_radius`: Calculates the particle radius that corresponds to the Kelvin curvature effect.
- `kelvin_term`: Computes the Kelvin term, defined as exp(kelvin_radius / particle_radius).

Specifically, the strategies differ in how they calculate:

- `effective_surface_tension`: Determines the effective surface tension of species based on their concentration.
- `effective_density`: Computes the effective density of species based on their concentration.

While each strategy may require additional parameters, defining surface tension is essential for all. The primary strategies include:

- `SurfaceStrategyMolar`: Uses mole fraction weighted values to determine surface tension and density.
- `SurfaceStrategyMass`: Uses mass fraction weighted values to determine surface tension and density.
- `SurfaceStrategyVolume`: Uses volume fraction weighted values to determine surface tension and density.

Each strategy is interchangeable and suitable for use in aerosol particle simulations. The choice of strategy should be guided by the available data and the level of detail required for the simulation.

In this tutorial, we will demonstrate how to create and use these strategies, employing builders and factories to instantiate them and calculate both the Kelvin radius and term.

In [1]:
import numpy as np
import matplotlib.pyplot as plt

from particula.next.particles.surface_builders import SurfaceStrategyMassBuilder
from particula.next.particles.surface_factories import SurfaceFactory
